In [29]:
from ast import literal_eval
from random import shuffle
import io
import itertools

import pandas as pd
import jsonlines

In [30]:
df = pd.read_csv('../data/working_dir/crowd_markup.csv')
df

,aspect,review,answers,message_id,filename,user_id
0,общее мнение о фильме,"Самое лучшее, что есть в этом фильме - это Ди ...","['Самое лучшее, что есть в этом фильме - это Д...",4914,981522130_20240514_19-53-54_ready_markup.csv,981522130
1,мнение о режиссёре,"Самое лучшее, что есть в этом фильме - это Ди ...",['-'],4917,981522130_20240514_19-53-54_ready_markup.csv,981522130
2,описание игры актёров,"Самое лучшее, что есть в этом фильме - это Ди ...",['Тоби Магуайр на эту роль не очень подходит в...,4920,981522130_20240514_19-53-54_ready_markup.csv,981522130
3,описание личности персонажей,"Самое лучшее, что есть в этом фильме - это Ди ...","['Нику Каррауэю тридцать лет, и он взрослый мы...",4923,981522130_20240514_19-53-54_ready_markup.csv,981522130
4,характеристика внешности персонажей,"Самое лучшее, что есть в этом фильме - это Ди ...",['-'],4926,981522130_20240514_19-53-54_ready_markup.csv,981522130
...,...,...,...,...,...,...
383,характеристика внешности персонажей,Фильм шииикааарный. После просмотра сказала се...,['-'],0,1247696527_20240511_01-01-01_ready_markup_OK.csv,1247696527
384,описание эмоций от просмотра фильма,Фильм шииикааарный. После просмотра сказала се...,['Фильм дает множество эмоций. Большинство из ...,0,1247696527_20240511_01-01-01_ready_markup_OK.csv,1247696527
385,описание декораций и графики,Фильм шииикааарный. После просмотра сказала се...,['Визуализация: Очень интересная интерпретация...,0,1247696527_20240511_01-01-01_ready_markup_OK.csv,1247696527
386,философские рассуждения,Фильм шииикааарный. После просмотра сказала се...,['-'],0,1247696527_20240511_01-01-01_ready_markup_OK.csv,1247696527


In [31]:
(df['answers'] == "['-']").sum()

163

In [32]:
from razdel import sentenize

In [33]:
def shorten_review(review: str, answers: set, symb_to_drop: int):
    print('Start sentence removal')
    from razdel import sentenize
    tokens_checked = 3

    sentences = list(sentenize(review))
    sentences = [sent.text for sent in sentences]

    reference = ' '.join(answers)

    sent_nums_to_drop = set()    

    for sent_num, sent in enumerate(sentences):
        tokens = sent.split()
        flags_to_drop = []
        for i in range(len(tokens) - (tokens_checked - 1)):
            if not all([tokens[i + j] in reference for j in range(tokens_checked)]):
                flags_to_drop.append(True)
            else:
                flags_to_drop.append(False)
                
        if all(flags_to_drop):
            sent_nums_to_drop.add(sent_num)
            print('Old symb_to_drop:', symb_to_drop)
            symb_to_drop -= len(sent)
            print('New symb_to_drop:', symb_to_drop)

            if symb_to_drop < 0:
                break
        else:
            print('Sentence exists in final answer')

    out_sentences = [sent for i, sent in enumerate(sentences) if i not in sent_nums_to_drop]

    print(f'{len(sentences) = } -> {len(out_sentences)}')
    print('Removed sentence:', sentences[list(sent_nums_to_drop)[0]])

    if symb_to_drop > 0:
        print(f'Not enough shortened ({symb_to_drop} left)')
    
    return ' '.join(out_sentences)


In [34]:
fp = open('../data/working_dir/yagpt_train.jsonl', 'wt', encoding='UTF-8')
writer = jsonlines.Writer(fp)

print(df.shape)
df = df.sample(frac=1.0)
print(df.shape)

for i in range(df.shape[0]):
    aspect = df.iloc[i]['aspect']
    review = df.iloc[i]['review']

    answers = df.iloc[i]['answers']
    answers = answers.replace('"', '\'')
    
    answers = answers.replace('[\'', '[`')
    answers = answers.replace('\']', '`]')
    answers = answers.replace('\', \'', '`, `')
    
    if '\'' in answers:
        answers = answers.replace('\'', '"')
        
    answers = answers.replace('[`', '[\'')
    answers = answers.replace('`]', '\']')
    answers = answers.replace('`, `', '\', \'')
    
    answers = list(
        literal_eval(answers)
    )

    for permutation in itertools.permutations(answers):    
        permutation = list(permutation)
        system_message = f'Использовать форматирование тебе можно только для списка. Говори коротко и используй только мнение из текста. Обязательно сохраняй исходный порядок слов. Можешь выделять конкретные фразы, а не предложения. \nТебе дана рецензия на фильм. Если в рецензии нет {aspect}, то ответь только "None" и игнорируй текст дальше. Если есть {aspect}, то выпиши из неё только эти части текста, в которых он встречается. (Пересказывать и вставлять комментарии запрещено, только выписывать имеющиеся части текста!) В ответе запиши список или просто "None".'
        user_message = review
        response = '* ' + '\n* '.join(permutation) if permutation != ['-'] else 'None'
    
        user_message = user_message.replace('        ', '\n')    
        while '  ' in user_message:
            user_message = user_message.replace('  ', ' ')        
        while '\n\n' in user_message:
            user_message = user_message.replace('\n\n', '\n')
    
        template = {"request": [{"role": "system", "text": ""}, {"role": "user", "text": ""}], "response": ""}
    
        EPS = 2
        len_request = len(user_message + system_message) + EPS
        if len_request > 4000:
            print('\n', len_request, sep='')
            user_message = shorten_review(user_message, permutation, len_request - 4000)
    
        template['request'][0]['text'] = system_message
        template['request'][1]['text'] = user_message
        template['response'] = response
    
        
        writer.write(template)

    # print(template, end='\n\n')


fp.close()

(388, 6)
(388, 6)

4095
Start sentence removal
Old symb_to_drop: 95
New symb_to_drop: 57
Old symb_to_drop: 57
New symb_to_drop: -6
len(sentences) = 36 -> 34
Removed sentence: Очень-очень долго я ждал эту премьеру.

4095
Start sentence removal
Old symb_to_drop: 95
New symb_to_drop: 57
Old symb_to_drop: 57
New symb_to_drop: -6
len(sentences) = 36 -> 34
Removed sentence: Очень-очень долго я ждал эту премьеру.

4361
Start sentence removal
Old symb_to_drop: 361
New symb_to_drop: 270
Old symb_to_drop: 270
New symb_to_drop: -33
len(sentences) = 29 -> 27
Removed sentence: Королевство полной луны – самый настоящий фильм для души, как бы старомодно это не звучало.

4101
Start sentence removal
Old symb_to_drop: 101
New symb_to_drop: 63
Old symb_to_drop: 63
New symb_to_drop: 0
Old symb_to_drop: 0
New symb_to_drop: -121
len(sentences) = 36 -> 33
Removed sentence: Очень-очень долго я ждал эту премьеру.

4101
Start sentence removal
Old symb_to_drop: 101
New symb_to_drop: 63
Old symb_to_drop: 63
New s